In [2]:
import os

import pandas as pd
import Python as py

In [3]:
root = os.getcwd()
df_scheduled = pd.read_csv(root + "/data/scheduled_loan_repayments.csv")
df_actual = pd.read_csv(root + "/data/actual_loan_repayments.csv")
df_balances = py.calculate_df_balances(df_scheduled, df_actual)

In [3]:
## Question 1
# print(df_balances[df_balances['ActualRepayment'] < df_balances['ScheduledRepayment']])
filtered_df = df_balances[df_balances['ActualRepayment'] < df_balances['ScheduledRepayment']]
count_distinct_loans_type1default = filtered_df['LoanID'].nunique()
count_distinct_loans = df_balances['LoanID'].nunique()
default_rate_percent = 100 * count_distinct_loans_type1default / count_distinct_loans
print(default_rate_percent)
print(py.question_1(df_balances))

15.0
15.0


In [4]:
# What timeframes are we dealing with?
print(max(df_balances["Month"])) # max month = 12 
print(min(df_balances["Month"])) # min month = 12
## First need to calculcate how many payments are expected
# print(df_balances[df_balances["ActualRepayment"] == 0].head())

## Check all scheduled payments for each loan 
repayments_per_loan = df_scheduled.groupby("LoanID").size()

## Unpaid payments are defined a payment being 0
unpaid_payments = df_balances[df_balances["ActualRepayment"] == 0].groupby("LoanID").size()

## Create a new DF that is on a loan-grain
loans_df = pd.DataFrame({
    'TotalExpectedRepayments': repayments_per_loan,
    'UnpaidPayments': unpaid_payments
})

## Update NaN values to 0 for calculcation purposes:
loans_df['UnpaidPayments'] = loans_df['UnpaidPayments'].fillna(0).astype(int)

## Calculate percentage of payments in a loan that were not repaid:
loans_df["PercentUnpaidPayments"] = 100 * loans_df["UnpaidPayments"] / loans_df["TotalExpectedRepayments"]

## Calculate what % of loans are in type 2 default:
default_rate_percent = 100 * len(loans_df[loans_df["PercentUnpaidPayments"] > 15]) / len(loans_df)
print(default_rate_percent)

12
1
15.0


In [4]:
## Question 3
## SMM is calculated as: (Unscheduled Principal)/(Start of Month Loan Balance)

## Aggregate portfolio by taking the sums of relevant columns per-month
portfolio_months = df_balances.groupby("Month").agg({
    "ActualRepayment": "sum",
    "ScheduledRepayment": "sum",
    "LoanBalanceStart": "sum"
})

## Unscheduled principal will be any payment towards the principal over what is expected
## This will be any payment over and above the expected payment.
portfolio_months["UnscheduledPrincipal"] = portfolio_months["ActualRepayment"] - portfolio_months["ScheduledRepayment"]
## UnscheduledPrincipal cannot be less than zero:
portfolio_months.loc[portfolio_months["UnscheduledPrincipal"] < 0, "UnscheduledPrincipal"] = 0

## SMM is calculated as: (Unscheduled Principal)/(Start of Month Loan Balance)
portfolio_months["SMM"] = portfolio_months["UnscheduledPrincipal"] / portfolio_months["LoanBalanceStart"]
print(portfolio_months["SMM"].describe())
print(portfolio_months[portfolio_months["UnscheduledPrincipal"] > 0].head())
print(portfolio_months["SMM"].max())

## SMM_mean is calculated as (∏(1+SMM))^(1/12) - 1
SMM_mean = (1 + portfolio_months["SMM"]).prod() ** (1/12) - 1
print(SMM_mean)

## CPR is calcualted as: 1 - (1- SMM_mean)^12
cpr = 100 * (1 - (1 - SMM_mean) ** 12)
print(cpr)
print(py.question_3(df_balances))

count    12.000000
mean      0.002325
std       0.000514
min       0.001520
25%       0.001869
50%       0.002403
75%       0.002703
max       0.003173
Name: SMM, dtype: float64
       ActualRepayment  ScheduledRepayment  LoanBalanceStart  \
Month                                                          
1           3114459.27          2997112.72       64950000.00   
2           3123642.10          2997112.72       62376790.69   
3           3109890.93          2997112.72       59772955.30   
4           3145514.81          2997112.72       57161172.37   
5           3146576.15          2997112.72       54492000.55   

       UnscheduledPrincipal       SMM  
Month                                  
1                 117346.55  0.001807  
2                 126529.38  0.002028  
3                 112778.21  0.001887  
4                 148402.09  0.002596  
5                 149463.43  0.002743  
0.003173342165950313
0.002324851754791135
2.75442453438689
2.75442453438689


In [6]:
## Question 4
## probability_of_default is taken from question 1 or question 2 answer
## Assume a recovery rate of 80%


## Need to calculate the total loan balance at the end of year 1
## To do this, sum all the loan balances at the end of month 12:
yearend_loan_balance = df_balances.loc[df_balances["Month"] == 12, "LoanBalanceEnd"].sum()

## Recovery rate is given to be 0.8
recovery_rate = 0.8

## Probabiltiy of default is taken from question 2
probability_of_default = 0.1

## Predicted total loss: probability_of_default * total_loan_balance * (1 - recovery_rate)
total_loss = probability_of_default * yearend_loan_balance * (1 - recovery_rate)
print(py.question_4(df_balances))

979573.1690999997
